In [1]:
import heapq
from enum import Enum
import copy

In [2]:
class Direction(Enum):
    buy = 0
    sell = 1

In [3]:
class Order(object):

    def __init__(self, order_id, quantity, broker, timestamp, direction):
        self.order_id = order_id
        self.quantity = quantity
        self.broker = broker
        self.timestamp = timestamp
        self.direction = direction

    def get_order_id(self):
        return self.order_id

    def get_broker(self):
        return self.get_broker()

    def get_timestamp(self):
        return self.timestamp

    def get_direction(self):
        return self.direction

    def get_quantity(self):
        return self.quantity

    def set_quantity(self, value):
        self.quantity = value

In [4]:
class LimitOrder(Order):
    def __init__(self, order_id, quantity, broker, timestamp, direction, price):
        Order.__init__(self, order_id, quantity, broker, timestamp, direction)
        self.price = price

    def signed_price(self):
        if self.direction is Direction.buy.value:
            return self.price
        else:
            # Transform Sell prices.
            return -self.price

    # Overload the unary negative (minus) operator.
    # To create a max-heap from a min-heap algorithm in 'heapq'
    def __neg__(self):
        return LimitOrder(self.order_id, self.quantity, self.broker, -self.timestamp, self.direction, -self.price)

    def __gt__(self, other):
        if( self.signed_price() > other.signed_price() ):
            return True
        elif( self.price == other.price ):
            if(self.timestamp < other.timestamp):
                return True
        return False

    def __eq__(self, other):
        return (self.price == other.price) and (self.timestamp == other.timestamp)

    def __lt__(self, other):
        return not ( self.__gt__(other) or self.__eq__(other) )

    def __str__(self):
        return "order_id: "  + str(self.order_id)  + ", " \
               "quantity: "  + str(self.quantity)  + ", " \
               "broker: "    + str(self.broker)    + ", " \
               "timestamp: " + str(self.timestamp) + ", " \
               "direction: " + str(self.direction) + ", " \
               "price: "     + str(self.price)

    def get_price(self):
        return self.price

In [5]:
class PriorityQueue(object):
    # Implements a max-heap (priority queue DS) based on 'heapq'
    # 'heapq' by default implements a min-heap
    # We are interested in the max-heap
    def __init__(self):
        self.heap = []

    def push(self, element):
        # This only works if '-' operator is overloaded for specialized types
        # Notice the '-' sign
        heapq.heappush( self.heap, -element )

    def pop(self):
        # Notice the '-' sign
        return -heapq.heappop( self.heap )

    def empty(self):
        if( not self.heap ):
            return True
        else:
            return False

    def top(self):
        # Notice the '-' sign
        if( not self.empty() ):
            return -self.heap[0]
        # Otherwise return None.

In [6]:
class OrderBook(object):

    def __init__(self):
        self.bids = PriorityQueue()
        self.asks = PriorityQueue()
        self.order_index = dict()

    def add(self, lo):
        
        # Duplicate order_id, means Update order
        # LO with quantity = 0, means Cancel order
        # LO with price = 0, means Market order
        # Duplicate order_ids must have duplicate brokers (as expected)
        # These features however are not implemented in this version
        # We are also not going to update the 'self.order_index' in this exercise
        
        if(lo.get_direction() == Direction.buy.value):
            self.bids.push( lo )
        else:
            self.asks.push( lo )
        self.match()

    # Credit to Yan Sun
    # This matching engine should work in a case where there is No duplicate order_id and where price and quantity are greater than 0.
    def match(self):
        if( not self.asks.empty() and not self.bids.empty() ):
            if (self.bids.top().get_price() >= self.asks.top().get_price()):
                if (self.bids.top().get_quantity() > self.asks.top().get_quantity()):
                    updated_volume = self.bids.top().get_quantity() - self.asks.top().get_quantity()

                    temp = self.bids.top()
                    temp.set_quantity(updated_volume)
                    self.bids.pop()
                    self.bids.push( temp )

                    self.asks.pop()
                    # Recursive
                    self.match()

                elif (self.bids.top().get_quantity() < self.asks.top().get_quantity()):
                    updated_volume = self.asks.top().get_quantity() - self.bids.top().get_quantity()

                    temp = self.asks.top()
                    temp.set_quantity(updated_volume)
                    self.asks.pop()
                    self.asks.push(temp)

                    self.bids.pop()
                    # Recursive
                    self.match()

                else:
                    self.bids.pop()
                    self.asks.pop()

    def best_ask(self):
        # Returns a type LimitOrder
        if( not self.asks.empty() ):
            return self.asks.top()

    def best_bid(self):
        # Returns a type LimitOrder
        if( not self.bids.empty() ):
            return self.bids.top()

    def spread(self):
        if (not self.bids.empty() and not self.asks.empty()):
            # Spread is always non-negative
            assert self.asks.top().get_price() - self.bids.top().get_price() >= 0
            return self.asks.top().get_price() - self.bids.top().get_price()

    def mid_price(self):
        if( not self.bids.empty() and not self.asks.empty() ):
            return (self.asks.top().get_price() + self.bids.top().get_price())/2.0

    def micro_price(self):
        # Volume Weighted Price
        if( not self.bids.empty() and not self.asks.empty() ):
            best_ask_price  = self.asks.top().get_price()
            best_ask_volume = self.asks.top().get_quantity()
            best_bid_price  = self.bids.top().get_price()
            best_bid_volume = self.bids.top().get_quantity()
            return ( best_bid_volume/(best_ask_volume + best_bid_volume) )*best_ask_price \
                   + ( best_ask_volume/(best_ask_volume + best_bid_volume) )*best_bid_price

    def info(self, order_id):
        if order_id in self.order_index:
            return self.order_index[order_id]
        # Otherwise return None.


    def show_state(self):
        print("\n")
        print("Buy-side (Bid) priority queue:")
        temp = copy.deepcopy(self.bids)
        # Otherwise, it will reference to the same object
        # and 'temp.pop()' will also pop the element in 'self.bids'
        while (not temp.empty()):
            print(temp.top())
            temp.pop()
        print("\n")

        print("Sell-side (Ask) priority queue:")
        temp = copy.deepcopy(self.asks)
        while (not temp.empty()):
            print(temp.top())
            temp.pop()
        print("\n")
        
        print("Order Book statistics:")
        print( "Spread:      " + str(self.spread())      + " £")
        print( "Mid price:   " + str(self.mid_price())   + " £")
        print( "Micro price: " + str(self.micro_price()) + " £")
        print("\n")

In [7]:
# Test 1) Check comparison operators on LOs

# LO Buy
a = LimitOrder(1016,4,'D',13,0,8726)
b = LimitOrder(1002,83,'O',16,0,10876)
print( a > b ) # Should be False
print( a < b ) # Should be True

# LO Sell
a = LimitOrder(1116,34,'M',3,1,9872)
b = LimitOrder(1001,31,'A',6,1,7823)
print( a > b ) # Should be False
print( a < b ) # Should be True

# LO Sell with equal price
a = LimitOrder(1116,34,'M',3,1,9872)
b = LimitOrder(1001,31,'A',6,1,9872)
print( a < b ) # Should be False, a joined the queue earlier
print( a == b ) # Should be False

False
True
False
True
False
False


In [8]:
# Test 1) Print the state of the LOB event-by-event


# 0 Buy Order, 1 Sell Order
# Orders are sorted by timestamp 
orders = [
    LimitOrder('1019', 62, 'L', 1, 1, 10702),
    LimitOrder('1006', 8, 'R',1, 1, 10665),
    LimitOrder('1004', 74, 'C', 2, 0, 9092),
    LimitOrder('1012', 92, 'H', 2, 1, 9684),
    LimitOrder('1005', 83, 'X', 4, 1, 9841),
    LimitOrder('1017', 89, 'D', 5, 1, 9784),
    LimitOrder('1001', 40, 'R', 5, 0, 9521),
    LimitOrder('1007', 19, 'N', 5, 1, 10388),
    LimitOrder('1013', 97, 'J', 6, 0, 9147),
    LimitOrder('1010', 41, 'R', 7, 0, 10572),
    LimitOrder('1015', 94, 'G', 8, 0, 10077),
    LimitOrder('1003', 91, 'Q', 8, 1, 8695),
    LimitOrder('1009', 59, 'S', 10, 0, 11066),
    LimitOrder('1018', 68, 'F', 12, 0, 8225),
    LimitOrder('1008', 3, 'K', 12, 1, 9849),
    LimitOrder('1016', 4, 'D', 13, 0, 8726),
    LimitOrder('1002', 83, 'O', 16, 0, 10876),
    LimitOrder('1011', 38, 'D', 18, 0, 11142),
    LimitOrder('1014', 54, 'Q', 19, 1, 9442),
    LimitOrder('1000', 68, 'S', 20, 1, 9287)
]


book = OrderBook()
index = 0
for order in orders:
    book.add(order)
    # Notice that the matching engine is Not implemented !!
    # Print the state of the LOB in real time (i.e. event by event)
    print("---------------------------------------------")
    print("Event: [" + str(index) + "]")
    index += 1
    print("---------------------------------------------")
    book.show_state()

---------------------------------------------
Event: [0]
---------------------------------------------


Buy-side (Bid) priority queue:


Sell-side (Ask) priority queue:
order_id: 1019, quantity: 62, broker: L, timestamp: 1, direction: 1, price: 10702


Order Book statistics:
Spread:      None £
Mid price:   None £
Micro price: None £


---------------------------------------------
Event: [1]
---------------------------------------------


Buy-side (Bid) priority queue:


Sell-side (Ask) priority queue:
order_id: 1006, quantity: 8, broker: R, timestamp: 1, direction: 1, price: 10665
order_id: 1019, quantity: 62, broker: L, timestamp: 1, direction: 1, price: 10702


Order Book statistics:
Spread:      None £
Mid price:   None £
Micro price: None £


---------------------------------------------
Event: [2]
---------------------------------------------


Buy-side (Bid) priority queue:
order_id: 1004, quantity: 74, broker: C, timestamp: 2, direction: 0, price: 9092


Sell-side (Ask) priori